In [13]:

import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [14]:
data=pd.read_csv('./dataset_5secondWindow-Copy1.csv')

In [16]:
data.head(10)

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,16170,78,NaN,100,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.000000,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.000000,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.00,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,15871,145,NaN,100,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.000000,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,16811,150,NaN,100,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.000000,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.000000,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,15831,105,NaN,77,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.000000,0.0,0.0,NaN,1.488361,0.575910,2.4158